In [ ]:
# Need to restart after:
!pip install convokit[llm]
!pip install convokit

In [ ]:
!git clone https://github.com/Sharp-4rth/temporal_belief_analysis.git

In [1]:
import sys
import os

os.chdir('/content/temporal_belief_analysis/notebooks')
print("Changed working directory to:", os.getcwd())

# Absolute path to src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

Changed working directory to: /content/temporal_belief_analysis/notebooks


In [2]:
import time
!pip install gdown
import zipfile
import nltk
from nltk.corpus import stopwords
from convokit import Corpus, download
import convokit
from temporal_belief.core.timeline_building import TimelineBuilder

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Download and unzip with python (Dataloading):
# !gdown "https://drive.google.com/file/d/1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1DLFY6JLMZqNjwvNRZmhlV4-rnoQP_eyH/view?usp=sharing" -O "/content/temporal_belief_analysis/merged_corpus_checkpoint_5.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/15NMRXEkGRoGjK6TXFBHIMOPjkTyZ0keP/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances200000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/15nVf6Js0KsDxA9HaB0zCXhc8VdcC-Fy-/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances75000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1dOUvQmtjFrXq0hJvnOsP_ZLqdGEyOVNJ/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/10HNQHIsFe386oBu_o5s9JOuo-wiaofXC/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances150000_llm.zip" --fuzzy
!gdown "https://drive.google.com/file/d/1nWaj5N8nsG7u5homv_kAh4CLPDv01M_Z/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_topics.zip" --fuzzy

# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/merged_corpus_checkpoint_5.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances200000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances75000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances150000_llm.zip").extractall("/content/temporal_belief_analysis")
zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_topics.zip").extractall("/content/temporal_belief_analysis")

Downloading...
From (original): https://drive.google.com/uc?id=1nWaj5N8nsG7u5homv_kAh4CLPDv01M_Z
From (redirected): https://drive.google.com/uc?id=1nWaj5N8nsG7u5homv_kAh4CLPDv01M_Z&confirm=t&uuid=118793da-003b-41f5-9571-5fd97fe192da
To: /content/temporal_belief_analysis/pd_corpus_with_topics.zip
100% 833M/833M [00:12<00:00, 67.1MB/s]


In [7]:
!gdown "https://drive.google.com/file/d/1hgSaV4VgTMzkQ9omjJHVnd-OJkA8UGxX/view?usp=sharing" -O "/content/temporal_belief_analysis/domain_adapted_model.zip" --fuzzy
zipfile.ZipFile("/content/temporal_belief_analysis/domain_adapted_model.zip").extractall("/content/temporal_belief_analysis")

Downloading...
From (original): https://drive.google.com/uc?id=1hgSaV4VgTMzkQ9omjJHVnd-OJkA8UGxX
From (redirected): https://drive.google.com/uc?id=1hgSaV4VgTMzkQ9omjJHVnd-OJkA8UGxX&confirm=t&uuid=ea43ae6d-3119-4a50-b26d-43c016804c3e
To: /content/temporal_belief_analysis/domain_adapted_model.zip
100% 481M/481M [00:07<00:00, 67.8MB/s]


In [4]:
corpus = Corpus(filename="/content/temporal_belief_analysis/pd_corpus_with_topics")

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem


In [17]:
# === Stance detection with your fine-tuned model + ConvoKit integration ===
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from tqdm import tqdm

# If you're already in a notebook where 'Corpus' is available, keep this import.
# Otherwise, uncomment the next line and ensure convokit is installed.
# from convokit import Corpus

# ---------- CONFIG ----------
MODEL_PATH = "/content/temporal_belief_analysis/stance-clf-best-domain-adapted"   # or "username/your-model"
MAX_LENGTH = 128                                    # speed/accuracy tradeoff
BATCH_SIZE = 32                                     # adjust to your VRAM
SAVE_PATH  = None                                   # e.g., "/content/corpus_with_stance"
FILTER_REMOVED = True                               # skip removed/deleted/empty
# ----------------------------

# Device selection (CUDA > MPS > CPU)
if torch.cuda.is_available():
    device = 0
    torch_dtype = torch.float16   # fp16 on GPU if supported
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
    torch_dtype = None
else:
    device = -1
    torch_dtype = None

# Load tokenizer/model and pipeline
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, torch_dtype=torch_dtype)

clf = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device,
    return_all_scores=True,
    truncation=True
)

def is_valid_text(txt: str) -> bool:
    if not txt:
        return False
    s = txt.strip()
    if FILTER_REMOVED and (s in ("[removed]", "[deleted]") or s == "."):
        return False
    return len(s) > 0

def run_batched_stance(corpus, batch_size=BATCH_SIZE, max_len=MAX_LENGTH, save_path=SAVE_PATH, max_utts=0):
    # Collect utterances & texts to score
    utts = []
    texts = []
    utts = list(corpus.iter_utterances())
    utts = utts[:max_utts] if max_utts > 0 else utts
    for utt in utts:
        if is_valid_text(utt.text):
            utts.append(utt)
            texts.append(utt.text)

    if not texts:
        print("No valid utterances to process.")
        return

    print(f"Scoring {len(texts)} utterances (batch_size={batch_size}, max_length={max_len})...")

    # Run batched inference
    # The pipeline accepts a list; we add padding/truncation and a max_length cap for speed.
    all_results = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Batches"):
        batch = texts[i:i+batch_size]
        res = clf(
            batch,
            batch_size=batch_size,
            padding=True,
            truncation=True,
            max_length=max_len
        )
        # `res` is List[List[{'label','score'}]] of length len(batch)
        all_results.extend(res)

    # Attach results back to utterances
    for utt, scores in zip(utts, all_results):
        # scores: [{'label': 'neutral', 'score': ...}, ...]
        # sort descending, get top
        scores_sorted = sorted(scores, key=lambda d: d["score"], reverse=True)
        top = scores_sorted[0]
        per_label = {d["label"]: float(d["score"]) for d in scores_sorted}

        utt.add_meta("detected_stance", top["label"])
        utt.add_meta("stance_confidence", float(top["score"]))
        utt.add_meta("stance_scores", per_label)
        # optional tag to record which model produced these
        utt.add_meta("stance_model", MODEL_PATH)

    if save_path:
        corpus.dump(save_path)
        print(f"Saved processed corpus to: {save_path}")

# ---- Usage example ----
# corpus = Corpus("/path/or/url")


Device set to use cuda:0


In [16]:
import time, math, random

texts = [utt.text for utt in corpus.iter_utterances() if utt.text]

# pick a representative subset of your texts
SAMPLE_N = 10000  # 10k usually enough for a stable rate
subset = random.sample(texts, min(SAMPLE_N, len(texts)))

# warmup (GPU + graph compile, caches)
_ = clf(subset[:512], batch_size=32, padding=True, truncation=True, max_length=128)

t0 = time.time()
_ = clf(subset, batch_size=32, padding=True, truncation=True, max_length=128)
dt = time.time() - t0

rate = len(subset) / dt                 # utterances per second
total = 4_000_000
eta_hours = (total / rate) / 3600

print(f"Throughput: {rate:.1f} utt/s")
print(f"ETA for 4,000,000: {eta_hours:.1f} hours")

Throughput: 661.6 utt/s
ETA for 4,000,000: 1.7 hours


In [18]:
run_batched_stance(corpus, batch_size=32, max_len=128, save_path="/content/temporal_belief_analysis", max_utts=5000)

KeyboardInterrupt: 